#Sistemas de recomendación y Filtrado Colaborativo

Importar las librerías necesarias para realizar el las operaciones de filtrado colaborativo.

In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds

Esta función normalizará la entrada pred_ratings parámetros:
  pred_ratings (calificaciones de predicción)



In [ ]:
def normalize(pred_ratings):
  return (pred_ratings - pred_ratings.min()) / (pred_ratings.max() - pred_ratings.min())

Esta función calculará la descomposición de valor único de la matriz de entrada
  dados n_factores. Luego generará y normalizará las predicciones de calificación de los usuarios.

  **Parámetros:**

*   **mat (CSR Matrix)**: matriz scipy csr correspondiente a la tabla dinámica (pt_df) 
*   **pt_df (DataFrame)**: marco de datos de pandas que es una tabla dinámica
*   **n_factors (Integer)** : Número de valores singulares y vectores a calcular.


Debe ser 1<= n_factors < min(mat.shape).

In [ ]:
def generate_prediction_df(mat, pt_df, n_factors):

  if not 1 <= n_factors < min(mat.shape):
   raise ValueError("Must be 1 <= n_factors < min(mat.shape)")
  
  # Factorización de matrices
  u, s, v = svds(mat, k = n_factors)
  s = np.diag(s)

  # Calcular las calificaciones de pred
  pred_ratings = np.dot(np.dot(u, s), v)
  pred_ratings = normalize(pred_ratings)

  # convertir a df
  pred_df = pd.DataFrame(pred_ratings,columns = pt_df.columns, index = list(pt_df.index)).transpose()

  return pred_df

Dado un usr_id y pred_df, esta función recomendará elementos al usuario.
     
**Parámetros:**
      
*   **pred_df (DataFrame)**: generado a partir de la función generate_prediction_df
*   **usr_id (Integer)** : El usuario para el que desea obtener recomendaciones de artículos
*   **n_recs (Integer)** : El número de recomendaciones que desea para este usuario



In [ ]:
def recommend_items(pred_df, usr_id, n_recs):

  usr_pred = pred_df[usr_id].sort_values(ascending = False).reset_index().rename(columns = {usr_id : 'sim'})
  rec_df = usr_pred.sort_values(by = 'sim', ascending = False).head(n_recs)
  return rec_df

In [ ]:
if __name__ == '__main__':
  # constantes
  PATH = 'data.csv'
  #importar datos
  df = pd.read_csv(PATH)
  print(df.shape)
  # generar una tabla dinámica con lectores en el índice y libros en la columna y los valores sean las calificaciones
  pt_df = df.pivot_table(columns = 'book_id',index = 'reader_id', values = 'book_rating').fillna(0)
  # convertir a una matriz csr
  mat = pt_df.values
  mat = csr_matrix(mat)
  pred_df = generate_prediction_df(mat, pt_df, 10)
  # generar recomendaciones
  print(recommend_items(pred_df, 1000, 5))

(100000, 10)
   book_id       sim
0     1341  0.153466
1     1516  0.148940
2     1839  0.147120
3     1108  0.145674
4     1025  0.145550
